In [1]:
from dotenv import load_dotenv
from glob import glob
from pprint import pprint #json 형식 출력에 편리
import os

In [2]:
load_dotenv()

True

In [ ]:
# chroma_db 읽기, 임베딩 하기

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model='text-embedding-3-small' # 실제 프로젝트에서는 large 사용
)

# db를 읽기 위한 목적이라, texts를 따로 넣어주지 않음!
vectorstore = Chroma(
    # 왜 임베딩이 필요할까? : 의미론적인 의미에서 검색하기 위해서, 즉 유사도를 비교하기 위해서, 임베딩으로 벡터 차원 공간에 문서를 박아넣음
    embedding_function=embeddings, 
    collection_name="chroma_test",     
    persist_directory='./chroma_db'
)

print(f"벡터 저장소에 저장된 문서 수 : {vectorstore._collection.count()}")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


벡터 저장소에 저장된 문서 수 : 14


failed 라고 발생한 것은 원격에서 접속할 때 발생하는 문제, 로컬 db에서는 문제 없음

## Prompt + LLM

In [ ]:
# Prompt + LLM
# langchain에서 openAI를 부를 때 사용하는 방식

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI( 
    model = 'gpt-4o-mini',
    temperature = 0.3,
    max_tokens = 100,
)

# langchain에서 메세지를 넣어주는 방법 [리스트 안 (튜플 형태로)]
messages = [
    ("system", "You are a helped assistant"),
    ("user", "{query}")
]

# chatPromptTemplate으로 형식을 만들기
prompt = ChatPromptTemplate.from_messages(messages=messages)
print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helped assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [9]:
prompt_text = prompt.format(query = "테슬라 창업자는 누구인가요?") #template안에 {query} 부분에 내용 넣기

print(prompt_text) #template이 어떻게 만들어졌는지 확인하기

System: You are a helped assistant
Human: 테슬라 창업자는 누구인가요?


In [10]:
response = llm.invoke(prompt_text)

print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하면서 CEO로 취임하게 되었고, 회사의 비전과 방향성을 이끌어왔습니다.


현재 결과는 chroma_db를 참고하지 않음

**정말 중요한 LCEL**

In [ ]:
# LCEL 체인을 구성 (정말 중요하다!!)
# '|' 연산자로 둘을 체인으로 연결

chain = prompt | llm

In [12]:
response =chain.invoke({"query" : "테슬라 창업자는 누구인가요?"})

print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey B. Straubel) 등 여러 명이 있습니다. 그러나 엘론 머스크는 2004년에 테슬라에 투자하고 CEO로 취임한 이후 회사의 얼굴이 되었으며, 그의 비전과 리더십으로 테


혹은 입력 값이 1개 밖에 없다면 딕셔너리 형식 생략 가능

In [13]:
response =chain.invoke("테슬라 창업자는 누구인가요?")

print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(JB Straubel), 이안 라이트(Ian Wright) 등 여러 명이 있습니다. 하지만 엘론 머스크는 테슬라의 CEO로서 가장 잘 알려져 있으며, 회사의 성장과 발전에 큰 영향을 미쳤습니다. 테


## Prompt + LLM + Output parser

내가 원하는 방식으로 답변 형태를 뽑아내기

### 문자열 파서

In [ ]:
# Parser 중에서 가장 많이 쓰이는 String parser
from langchain_core.output_parsers import StrOutputParser

# 출력 파서를 생성

output_parser = StrOutputParser()
output_parser.invoke(response) # 문자열 부분만 가져오기

'테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(JB Straubel), 이안 라이트(Ian Wright) 등 여러 명이 있습니다. 하지만 엘론 머스크는 테슬라의 CEO로서 가장 잘 알려져 있으며, 회사의 성장과 발전에 큰 영향을 미쳤습니다. 테'

In [15]:
str_chain = prompt | llm | output_parser

query = "리비안의 설립년도는 언제인가요?"
str_response = str_chain.invoke(query)

print(str_response)

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 데 주력하고 있으며, 특히 전기 픽업트럭과 SUV 모델로 주목받고 있습니다.


### json parser

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

json_response = chain.invoke("테슬라 창업주는 누구인가요? Json 형식으로 출력해 주세요.")
print(json_response) #json 형식으로 나오지만, 여전히 활용하기 어렵고 보기 불편함


content='```json\n{\n  "창업주": {\n    "이름": "엘론 머스크",\n    "출생연도": 1971,\n    "국적": "남아프리카 공화국",\n    "직업": "기업가, 발명가"\n  }\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 34, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'finish_reason': 'stop', 'logprobs': None} id='run-00d4ef01-654b-4299-9b8a-b058ac3d368d-0' usage_metadata={'input_tokens': 34, 'output_tokens': 65, 'total_tokens': 99}


In [17]:
json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

{'창업주': {'이름': '엘론 머스크', '출생연도': 1971, '국적': '남아프리카 공화국', '직업': '기업가, 발명가'}}


### Pydatic 모델을 생성

LLM의 결과 값을 객체 형태로 만드는 방법

In [5]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel): #BaseModel 이 정확하게 뭐하는 역할인가?
    """Information about a person."""
    name : str = Field(...,description="The name of the person")
    title : str = Field(..., description="The title of position of the person")


person_parser = PydanticOutputParser(pydantic_object=Person)

print("==================================================")
print(person_parser.get_format_instructions()) # prompt 가 출력됨. parser에서 정의한 프롬프트
print("==================================================")

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "properties": {"name": {"title": "Name", "description": "The name of the person", "type": "string"}, "title": {"title": "Title", "description": "The title of position of the person", "type": "string"}}, "required": ["name", "title"]}
```


In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the user query, Wrap the output in 'json' tags\n {format_instructions}"),
        ("human", "{query}")
    ]
).partial(format_instructions=person_parser.get_format_instructions())

In [7]:
person_chain = prompt | llm | person_parser
response = person_chain.invoke("테슬라의 창업자는 누구인가요?")
response

Person(name='엘론 머스크', title='CEO 및 공동 창립자')

#### Chat Completion Method : invoke()
### Stream

In [8]:
import time

for chunk in llm.stream("테슬라 창업자는 누구인가요?"):
    print(chunk.content, end='', flush=True)
    time.sleep(0.1)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하여 회사의 성장에 큰 영향을 미쳤습니다. 이후 그는 테슬라의 얼굴이자 주요 추진력

### batch

In [9]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?"
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(JB Straubel), 이안 라이트(Ian Wright) 등 여러 명이 있습니다. 하지만 엘론 머스크는 테슬라의 CEO로서 가장 잘 알려져 있으며, 회사의 성장과 발전에 큰 영향을 미쳤습니다. 테

================================== Ai Message ==================================

리비안(Rivian)의 창업자는 RJ 스캐링(RJ Scaringe)입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 제조업체로서 SUV와 픽업트럭을 주로 생산하고 있습니다. 리비안은 지속 가능한 이동 수단을 제공하는 것을 목표로 하고 있습니다.



### 문서 검색기(Vector DB에서)

In [ ]:
# retriever

retriever = vectorstore.as_retriever(
    search_kwargs={'k':2}
)

query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query) # 이것은 List[document] 자료형이다!

retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])
pprint(retrieved_docs_text)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010\n'
 '명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를 차지했습니다.')


### 병렬처리하기

In [14]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

runnable = RunnableParallel(
    {
        "context" : itemgetter("context"),
        "question" : itemgetter("question")
    }
)

response = runnable.invoke({"context" : retrieved_docs_text, "question" : query})

response

{'context': '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010\n명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n\n2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 세계 전기차 시장의 약 12.9%를 차지했습니다.',
 'question': '테슬라 창업자는 누구인가요?'}

### RunnablePassthrough + lambda

RunnablePassthrough : 데이터 값을 그대로 전달하기 위해서

In [ ]:
from langchain_core.runnables import RunnablePassthrough

runnable = RunnableParallel(
    question = RunnablePassthrough()
)

runnable.invoke({"query": "테슬라 창업자는 누구인가요?"})

{'question': {'query': '테슬라 창업자는 누구인가요?'}}

In [17]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())

runnable = RunnableParallel(
    question = RunnablePassthrough(),
    word_count = RunnableLambda(count_num_words)
)

runnable.invoke("테슬라 창업자는 누구인가요?")

{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

## RAG 프롬프트 템플릿 파이프라인 구성

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """다음 context 만을 바탕으로 질문에 답하라.
외부 정보나 지식을 사용하지 말라.
답변이 context 와 맞지 않는 경우 혹은 일치하지 않는 경우 답변을 "잘 모르겠습니다." 라고 하라.

[Context]
{context}

[Question]
{question}

[Answer]
"""

# 위에서는 from_message로 템플릿을 만들었지만, 문자열로 템플릿을 만들 때에는 from_template을 사용한다.
prompt = ChatPromptTemplate.from_template(template=template)

prompt.pretty_print()

================================ Human Message =================================

다음 context 만을 바탕으로 질문에 답하라.
외부 정보나 지식을 사용하지 말라.
답변이 context 와 맞지 않는 경우 혹은 일치하지 않는 경우 답변을 "잘 모르겠습니다." 라고 하라.

[Context]
{context}

[Question]
{question}

[Answer]



In [19]:
retriever = vectorstore.as_retriever(search_kwargs={"k":2})

# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

retriever_chain = retriever | format_docs

response = retriever_chain.invoke("테슬라 창업자는 누구야?")

pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010\n'
 '\n'
 '명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를 차지했습니다.')


In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=100)

rag_chain = (
    {"context" : retriever_chain, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke(query)

In [21]:
response

'테슬라의 창업자는 마틴 에버하드와 마크 타페닝입니다.'